# 決算情報と株価情報をマージして、割高/割安 (高/低PBR) 銘柄の抽出

high_roa_companies.ipynb ではROAでソートしましたが、今回は株価情報をジョインしてPBR (株価純資産倍率) で集計します。
PBRは株の割安性を判断するために使われます。できれば純資産や負債額を考慮して割安性を判断したいですが、とりあえず自己資本対時価総額のPBRから見てみようと思います。

まずは同様にデータを取ります。

In [2]:
import pandas as pd

In [8]:
stock_data = pd.read_excel("./data/20161103f.xls")

In [9]:
stock_data.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [64]:
stock_data.head(3)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開又は更新日
0,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,5.510000e+08,5.25,4.84,2.299500e+10,1.016320e+11,NaN,NaN,NaN,NaN,2016-08-05
1,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,1.799000e+09,17.13,15.79,2.306500e+10,9.460800e+10,214.97,2.689000e+09,-5.114000e+09,2.482000e+09,2016-05-09
2,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,1.133000e+09,10.79,9.95,2.341700e+10,1.143630e+11,NaN,NaN,NaN,NaN,2016-02-05


## 株価情報をとる

株価情報は [k-db](http://k-db.com/stocks/) さんのCSVデータをお借りします。
`encoding` にはCP932を指定しましょう。

In [65]:
price_data = pd.read_csv("./data/stocks_2016-11-04.csv", encoding="cp932")

In [66]:
price_data.columns

Index(['コード', '銘柄名', '市場', '始値', '高値', '安値', '終値', '出来高', '売買代金'], dtype='object')

## カラム名の変更

ジョインするためにカラム「コード」を「証券コード」に寄せます。

In [67]:
price_data = price_data.rename(columns={"コード": "証券コード"})

In [68]:
price_data.columns

Index(['証券コード', '銘柄名', '市場', '始値', '高値', '安値', '終値', '出来高', '売買代金'], dtype='object')

## 証券コードの形式を揃える

ここでデータが文字列になっているので、末尾のコードを取って数値化します。
これで下準備は完了です。

In [69]:
price_data.head(3)

,証券コード,銘柄名,市場,始値,高値,安値,終値,出来高,売買代金
0,1301-T,極洋,東証1部,2765.0,2767.0,2686.0,2697.0,31500,85688600
1,1305-T,ETFTPX,東証,1402.0,1407.0,1389.0,1399.0,698420,975215520
2,1306-T,TOPIX投,東証,1385.0,1389.0,1370.0,1380.0,3489990,4810094170


In [76]:
price_data["証券コード"] = price_data["証券コード"].map(lambda x: int(x[:-2]))

In [77]:
price_data.head(3)

,証券コード,銘柄名,市場,始値,高値,安値,終値,出来高,売買代金
0,1301,極洋,東証1部,2765.0,2767.0,2686.0,2697.0,31500,85688600
1,1305,ETFTPX,東証,1402.0,1407.0,1389.0,1399.0,698420,975215520
2,1306,TOPIX投,東証,1385.0,1389.0,1370.0,1380.0,3489990,4810094170


## .mergeを使ってジョインする

SQL言うところのジョインは、Pandasでは `pandas.merge` だそうです。
マージの条件は `on=` に指定します。

In [192]:
data = pd.merge(stock_data, price_data, on="証券コード")

In [149]:
data.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日', '銘柄名', '市場', '始値', '高値', '安値',
       '終値', '出来高', '売買代金'],
      dtype='object')

## 確認する

データを確認してみると、ジョインできているようです。

In [80]:
data.head(3)

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目（売上高欄に掲載）,売上高,...,財務キャッシュフロー,情報公開又は更新日,銘柄名,市場,始値,高値,安値,終値,出来高,売買代金
0,1301,極洋,日本基準,連結,2017年3月期,第1四半期,2016-04-01,2016-06-30,売上高,5.220600e+10,...,NaN,2016-08-05,極洋,東証1部,2765.0,2767.0,2686.0,2697.0,31500,85688600
1,1301,極洋,日本基準,連結,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.266260e+11,...,2.482000e+09,2016-05-09,極洋,東証1部,2765.0,2767.0,2686.0,2697.0,31500,85688600
2,1301,極洋,日本基準,連結,2016年3月期,第3四半期,2015-04-01,2015-12-31,売上高,1.788900e+11,...,NaN,2016-02-05,極洋,東証1部,2765.0,2767.0,2686.0,2697.0,31500,85688600


## 不要な行の削除と、必要な列の追加

ここで例によって通期、連結の決算情報のみを絞り込みます。
あと、この決算情報には発行済み株式数が無かったので「純資産又は株主資本 / 一株あたり純資産」で求めておきます。

In [193]:
data = data[(data["決算期間"] == "通期") & (data["連結個別"] == "連結")]

In [194]:
data[["純資産又は株主資本", "一株当り純資産"]].head(3)

,純資産又は株主資本,一株当り純資産
1,2.306500e+10,214.97
5,2.306900e+10,215.65
9,1.993000e+10,187.57


In [195]:
data["発行済み株式数"] = (data["純資産又は株主資本"] / data["一株当り純資産"]).map(lambda x: abs(x))

## 時価総額、PBRの算出

時価総額は発行済み株式数 x 11月4日の終値で求めます。
PBRは時価総額 / 純資産又は株主資本で求めます。
決算期がまばらなので、11月4日の終値だけでみると若干微妙ではあります。

In [196]:
data["時価総額"] = data["発行済み株式数"] * data["終値"]

In [197]:
data["PBR"] = data["時価総額"] / data["純資産又は株主資本"]

## サマリーを作る

PBRが0.4以上で、見たいカラムをとりあえずとったDataFrameをサマリーとして作ります。
そこから `sort_values()` を使ってPBRでソート、上の数十件を取り出します。

In [198]:
pbr_summary = data[lambda r: r["PBR"] > 0.4][["証券コード", "企業名", "決算期", "PBR", "発行済み株式数", "時価総額", "純資産又は株主資本"]]

## 低PBR(割安)銘柄

In [199]:
pbr_summary.sort_values("PBR", ascending=True).head(10)

,証券コード,企業名,決算期,PBR,発行済み株式数,時価総額,純資産又は株主資本
29657,6337,テセック,2016年3月期,0.400396,5.648981e+06,3.332899e+09,8.324000e+09
43842,8104,クワザワ,2015年3月期,0.400597,8.276007e+06,4.419388e+09,1.103200e+10
47155,8524,北洋銀行,2016年3月期,0.400756,4.044550e+08,1.561196e+11,3.895630e+11
47386,8550,栃木銀行,2014年3月期,0.400795,1.158609e+08,5.514977e+10,1.376010e+11
30169,6382,トリニティ工業,2015年3月期,0.400880,1.853005e+07,8.375582e+09,2.089300e+10
48043,8705,岡藤ホールディングス,2014年3月期,0.400964,8.546743e+06,1.435853e+09,3.581000e+09
32565,6704,岩崎通信機,2015年3月期,0.401033,9.973081e+07,7.280349e+09,1.815400e+10
1899,1882,東亜道路工業,2016年3月期,0.401790,5.171278e+07,1.406588e+10,3.500800e+10
50232,9063,岡山県貨物運送,2015年3月期,0.402024,2.070067e+07,4.636950e+09,1.153400e+10
11000,3204,トーア紡コーポレーション,2014年12月期,0.402112,8.925157e+06,4.560755e+09,1.134200e+10


## 高PBR(割高)銘柄

In [202]:
pbr_summary.sort_values("PBR", ascending=False).head(30)

,証券コード,企業名,決算期,PBR,発行済み株式数,時価総額,純資産又は株主資本
4907,2315,ＳＪＩ,2015年10月期,197.222222,3.888889e+08,2.761111e+10,1.400000e+08
17445,4222,児玉化学工業,2016年3月期,164.705882,1.873529e+09,1.049176e+11,6.370000e+08
34971,6942,ソフィアホールディングス,2016年3月期,162.365591,4.193548e+07,6.332258e+09,3.900000e+07
47011,8462,フューチャーベンチャーキャピタル,2015年3月期,110.139050,1.373117e+08,2.610295e+11,2.370000e+09
15254,3832,Ｔ＆Ｃホールディングス,2014年11月期,106.756757,3.783784e+07,2.989189e+09,2.800000e+07
15422,3845,アイフリークホールディングス,2014年3月期,100.000000,1.383648e+07,2.200000e+09,2.200000e+07
29644,6336,石井表記,2015年1月期,64.869029,2.642527e+08,1.112504e+11,1.715000e+09
36041,7172,ジャパンインベストメントアドバイザー,2013年12月期,60.294688,1.023653e+07,3.183560e+10,5.280000e+08
47015,8462,フューチャーベンチャーキャピタル,2014年3月期,58.745365,1.129481e+08,2.147143e+11,3.655000e+09
9077,3010,価値開発,2014年3月期,49.049430,1.354880e+08,5.243384e+10,1.069000e+09


## 日経225銘柄のPBR

日経平均株価に登録されている銘柄からPBRを見てみようと思います。


In [114]:
nikkei225 = [7013, 8304, 3407, 5201, 2502, 2802, 4503, 6857, 6113, 6770, 8267, 7202, 8001, 4208, 4523, 9202, 6472, 9613, 9437, 6361, 8725, 6103, 9532, 3861, 1802, 6703, 9007, 7733, 4452, 6952, 1812, 9107, 7012, 9503, 2801, 7751, 6971, 4151, 2503, 6326, 3405, 8253, 9008, 9009, 9433, 5406, 1605, 9766, 4902, 6301, 1721, 7186, 2501, 3436, 6674, 5411, 5020, 6473, 3086, 4507, 8355, 4911, 7762, 1803, 9104, 5002, 4004, 4063, 8303, 5401, 9412, 7735, 7269, 5232, 4005, 5713, 6302, 8053, 5802, 8830, 1928, 9735, 3382, 2768, 6758, 8729, 9984, 8630, 4568, 8750, 6367, 1801, 7912, 4506, 5541, 5233, 6976, 1925, 8601, 8233, 2531, 4502, 8331, 4519, 9502, 6366, 8795, 2432, 3401, 6762, 4543, 4061, 6902, 4324, 5301, 9022, 3289, 8035, 8766, 9531, 9005, 8804, 9501, 9681, 6502, 4042, 5332, 9001, 9602, 5707, 5901, 3101, 3402, 5714, 4043, 7911, 7203, 8015, 4704, 7731, 9021, 2871, 1963, 4021, 7201, 5413, 2002, 3105, 6988, 5202, 5333, 4272, 5703, 1332, 6471, 3863, 5631, 4041, 2914, 9062, 6701, 5214, 9432, 2282, 9101, 8604, 6773, 1808, 6752, 9020, 6305, 6501, 7004, 7205, 9983, 6954, 8354, 5803, 7270, 6702, 6504, 4901, 5108, 5715, 5801, 3865, 7267, 8628, 7261, 8252, 1333, 8002, 8411, 4183, 5706, 8309, 8316, 7003, 8031, 8801, 3099, 4188, 7211, 7011, 8058, 9301, 8802, 6503, 5711, 8306, 6767, 6479, 2269, 6508, 6506, 4689, 9064, 7951, 7272, 8028, 3103, 6841, 5101, 7752, 8308]

### in で絞り込む

`__contains__` で絞り込む際 Pandas では `Series.isin` を使うそうです。
`Series.str.startswith` など絞り込みの条件はいくつかもってるようです。
`==` で結んだ時も内部的には `__eq__` をうまく使ったりして条件になるオブジェクトを返してるんでしょうね。

In [143]:
nikkei225_data = data[data["証券コード"].isin(nikkei225)]

## 日経225からの低PBR銘柄

日経225でかつ低PBRの銘柄を探します。
でもただ低PBRだと面白くないので、ROEが10%より高くて自己資本率が40%より高い銘柄を探します。

In [207]:
nikkei225_data["ROE"] = nikkei225_data["純利益"] / nikkei225_data["純資産又は株主資本"] * 100

/home/hirokiky/dev/pydata_now/env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [208]:
nikkei225_data["ER"] = nikkei225_data["純資産又は株主資本"] / nikkei225_data["総資産"] * 100

/home/hirokiky/dev/pydata_now/env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [212]:
nikkei225_data[(nikkei225_data["ER"] > 40) & (nikkei225_data["ROE"] > 10)].sort_values("PBR").head(30)[["証券コード", "企業名", "決算期", "ROE", "ER", "PBR"]]

,証券コード,企業名,決算期,ROE,ER,PBR
16494,4041,日本曹達,2016年3月期,10.885321,59.608680,0.544088
36233,7211,三菱自動車,2015年3月期,17.617172,42.378390,0.803297
23061,5101,横浜ゴム,2015年12月期,10.533294,48.498697,0.831556
23065,5101,横浜ゴム,2014年12月期,12.244318,45.034315,0.877868
2785,1963,日揮,2016年3月期,10.196748,60.841396,1.012980
52424,9531,東京ガス,2016年3月期,10.037555,49.529784,1.035299
2793,1963,日揮,2014年3月期,12.419120,50.915558,1.132528
25246,5714,ＤＯＷＡホールディングス,2016年3月期,10.732163,55.806487,1.179668
52432,9531,東京ガス,2014年3月期,10.534419,47.293478,1.182894
25258,5714,ＤＯＷＡホールディングス,2015年3月期,13.566642,51.596153,1.228847


## まとめ

以上です。
一般的なスクリーニングツールを使ってもそうですが、これである程度銘柄の検討をつけてから有価証券報告書を追っていくのが良いかなと思います。

ただまぁROAにしろPBRにしろまだ一般的なスクリーニングツールでできる範疇のことなので、今後はそれらでできないことができると良いなぁと思います。

とはいえこれだけ簡単にデータを修正して、JupyterNotebookで見れるのは楽しいですね。
フィンテックですよフィンテック！